In [1]:
import numpy as np
from shapely import Polygon, LineString
from shapely.ops import unary_union
import utm
import ast

In [2]:
def from_latlon(latitude, longitude):
    """
    convert lat, lon to (x, y) coordinates (in meters)
    :param latitude:  list or np.array (shape: (n,))
    :param longitude: list or np.array (shape: (n,))
    :return: np.array of (x, y) coordinates (shape: (n, 2))
    """
    xy = np.array([
        utm.from_latlon(lat, lon)[:2] for lat, lon in zip(latitude, longitude)
    ])
    return xy



def get_subfields_xy(field_bounds: list):
    """
    convert GPS bounds of field into (x, y) coordinates (in meters)
    :param field_bounds: list with GPS bounds of subfields
    :return: list of (x, y) bounds of the subfields
    """
    subfields = [np.array(subfield).T for subfield in field_bounds]
    bounds_xy = [from_latlon(field[1], field[0]) for field in subfields]
    return bounds_xy



def get_bounds_from_string(field_bounds_str : str):
    """
    convert string representation of the bounds into list
    :param field_bounds_str: string representation of the bounds
    :return: list of bounds of the subfields
    """
    if field_bounds_str[:4] == '[[[[':
        field_bounds_str = field_bounds_str.replace('[[[', '[[').replace(']]]', ']]')
    field_bounds = ast.literal_eval(field_bounds_str) 
    return field_bounds

In [3]:
def get_area_and_distance(
    tool_width : float,
    time : list,
    track_lat : list,
    track_lon : list,
    path_distance : list,
    field_processed : list,
    field_bounds : list
):
    """
    calculate path distance and processed field area
    :param tool_width: width of the tool (in meters)
    :param track_lat: list or np.array (shape: (n,))
    :param track_lon: list or np.array (shape: (n,))
    :param path_distance: list of pre-calculated path distances (shape: (m,))
    :param field_processed: list of pre-calculated processed area (shape: (m,))
    :param field_bounds: list of GPS subfields bounds
    :return: dictionary with calculated processed area and path distance
    """
    # convert GPS coordinates of the track
    track_xy = from_latlon(track_lat, track_lon)
    # convert GPS coordinates of the field bounds and create field polygon
    subfields_xy = get_subfields_xy(field_bounds)
    field_polygon = unary_union([Polygon(subfield) for subfield in subfields_xy])
    # calculate distance and area and add them to the lists
    for i in range(len(field_processed), len(time)):
        track = LineString(track_xy[:i])
        path_distance.append(track.length)
        
        buffered_track = track.buffer(tool_width/2)
        processed_step = field_polygon.intersection(buffered_track)
        field_processed.append(processed_step.area)
        
    # create dictionary with output
    area_calc = {
        'tool_width': tool_width,
        'time': time,
        'lat': track_lat,
        'long': track_lon,
        'path_distance': path_distance,
        'field_processed': field_processed
    }

    return area_calc

In [4]:
def update_area_calc(area_calc : dict, field_bounds_str : str):
    """
    update area_calc state 
    :param area_calc: dictionary of the field processing state
    :param field_bounds_str: string representation of the bounds
    :return: updated dictionary of the field processing state 
    
    """
    field_bounds = get_bounds_from_string(field_bounds_str)
    area_calc = get_area_and_distance(
        tool_width=area_calc['tool_width'],
        time=area_calc['time'],
        track_lat=area_calc['lat'],
        track_lon=area_calc['long'],
        path_distance=area_calc['path_distance'],
        field_processed=area_calc['field_processed'],
        field_bounds=field_bounds
    )
    return area_calc